<a href="https://colab.research.google.com/github/MoonEeSun/Artificial_Intelligence/blob/main/assignment/HW5_Cliff_Walking_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[인공지능] 과제5 Cliff Walking 예제 DQN 구현**
*   **DQN Class를 완성하여 결과를 살펴보는 것이 목표**입니다.
*   기본적인 코드는 아래 노트에 모두 작성되어 있습니다. 비어있는 함수 부분을 완성하면 됩니다.
*   **과제 수행 시 주의사항: 외부 라이브러리로 DQN 적용하지 말 것, 수업 때 배운 내용대로 DQN을 주어진 함수에 구현할 것.** 웹 상에 있는 다양한 DQN 코드를 참고하는 것은 괜찮습니다.
*   **보고서 작성 내용**: 여러분이 완성한 DQN 알고리즘의 내용과 결과의 의미를 분석하는 내용을 작성하면 됩니다.
작성한 코드와 실행 결과를 첨부하길 바라며, 코드에는 자세한 주석을 필수적으로 포함하기 바랍니다. 보고서는 PDF로 제출바랍니다.
*   보고서는 12월 16일 오후 11시 59분까지 블랙보드에 보고서 형태로 제출하면 됩니다. 지각은 0점입니다.
*   **Deep Sarsa를 추가로 구현하여 보고서에 관련 내용을 추가적으로 작성 및 제출할 경우 가산점 5점 부여**

# **본 노트를 본인의 drive로 복사하여 활용하기 바랍니다.**

본 과제 역시 이전과제와 동일한 환경을 사용합니다. 따라서 학습된 Q-value를 입력하면 해당하는 Q-value의 greedy 정책이 출력되도록 하는 QtoPolicy Class 또한 동일하게 사용됩니다.

In [ ]:
import numpy as np
import random
from tqdm import tqdm
from collections import defaultdict, namedtuple, deque
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv # Cliff Walking 환경

# 신경망 구현을 위한 pytorch 라이브러리 import
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# 학습된 Q-value를 입력하면 그 Q-value의 greedy 정책이 출력되도록 함
# DQN이나 Deep Sarsa를 통해 학습된 정책을 출력하기 위해 필요
class QtoPolicy:
    def __init__(self):
        self.action = ['↑', '→', '↓', '←', 'X'] # Agent의 가능한 Action

    def printPolicy(self, Q):
        # Agent의 정책 : 학습된 Q-value가 Q-table에 있으면 Q-value를 가장 크게 만드는 정책을 저장(greedy한 정책)
        policy = np.array([np.argmax(Q[key]) if key in Q else -1 for key in np.arange(48)])
        print('init policy=',policy)
        # 가장 큰 Q-value : 학습된 Q-value가 Q-table에 있으면 Q-value 중 max값을, 없으면 0을 변수 v에 저장
        v = ([np.max(Q[key]) if key in Q else 0 for key in np.arange(48)])
        print('value =',v)
        # Agent의 행동 저장
        actions = np.stack([self.action for _ in range(len(policy))], axis=0)
        print('actions =',actions)
        # cliff walking 초기 환경 구축
        policy[36:] = np.array([0] + [3] * 10 + [4])

        print(np.take(actions, np.reshape(policy, (4, 12))))
        print('')

DQN의 experience replay기법 구현을 위한 replay buffer class를 생성합니다. DQN에서는 (state, action, reward, next_state)가 저장됩니다.

In [ ]:
# Experience replay를 위한 replay buffer class 생성
# agent가 관찰한 transition을 저장 - (state, action) pair을 (next_state, reward) 결과에 매핑
Transition = namedtuple('Transition',
                        ('state', 'action', 'reward', 'next_state'))

# replay buffer
# bounded size의 cyclic buffer - 최근에 관찰한 transition을 가지고 있음
class ReplayBuffer(object):

    # 버퍼 초기화
    def __init__(self, capacity):
        self.buffer = deque([],maxlen=capacity)
    
    # 버퍼에 Agent의 경험 정보 추가
    def push(self, *args):
        self.buffer.append(Transition(*args))

    # .sample() 메쏘드 : 학습을 위해 transition의 랜덤한 batch를 선택
    # experience replay에서는 Agent의 경험 정보를 저장하는 replay buffer에서 
    # 경험정보를 sampling하여 신경망 update에 사용
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    # 버퍼 길이 계산
    def __len__(self):
        return len(self.buffer)

DQN에서 Q-Network으로 사용할 신경망 모델을 PyTorch 기반으로 정의합니다.

In [ ]:
# DQN은 Q-learning에서의 Q-table을 신경망으로 구성한 것
# DQN의 Q-Network를 신경망으로 구현하기 때문에 DNN 선언
class DNN(nn.Module):
  # 신경망 초기화
    def __init__(self, inputs, outputs):
        super(DNN, self).__init__() 
        self.x_dim = inputs # input 차원
        self.y_dim = outputs # output 차원
        self.fc_variable_no = 100 # 층 개수

        # network 용 변수
        self.fc_in = nn.Linear(self.x_dim, self.fc_variable_no) # 입력층
        self.fc_hidden1 = nn.Linear(self.fc_variable_no, self.fc_variable_no) # 은닉층1
        self.fc_hidden2 = nn.Linear(self.fc_variable_no, self.fc_variable_no) # 은닉층2
        self.fc_hidden3 = nn.Linear(self.fc_variable_no, self.fc_variable_no) # 은닉층3
        self.fc_out = nn.Linear(self.fc_variable_no, self.y_dim) # 출력충
        self.relu = nn.ReLU() # 활성함수 - ReLU(: DNN에서의 그레디언트 소멸 문제를 해결하기 위해 ReLU 활성함수 사용)
    
    # forward propagation
    def forward(self, x):
        x = torch.reshape(x, [-1, self.x_dim]) # 차원을 맞추기 위해 reshape
        x = self.relu(self.fc_in(x)) # 입력층 
        x = self.relu(self.fc_hidden1(x)) # 은닉층1
        x = self.relu(self.fc_hidden2(x)) # 은닉층2
        x = self.relu(self.fc_hidden3(x)) # 은닉층3
        x = self.fc_out(x) # 출력층
        return x

DQN 알고리즘 class를 정의합니다. 하이퍼파라미터는 주어진 값을 사용하면 됩니다.

In [ ]:
# DQN 알고리즘 class
class DQN:
    def __init__(self):
        self.state_no = 48 # 상태 수
        self.action_no = 4 # action 수
        self.alpha = 0.001 # learning rate
        self.gamma = 0.99 # discount vector
        self.epsilon = 0.2 # 앱실론

        self.batch_size = 32  # Experience replay에서의 batch size
        self.training_interval = 10  # main Q-Network 학습 interval
        self.target_update_interval = 100  # target Q-Network 학습 interval
        # 100번에 한번씩 target계산을 위한 target Q-Network를 main Q-Network로 update

        # fixed target Q-value를 위한 main_network, target_network
        self.main_net = DNN(self.state_no, self.action_no) 
        self.target_net = DNN(self.state_no, self.action_no) # target 계신용

        # target Q-network를 main Q-network와 동일하게 초기화
        self.target_net.load_state_dict(self.main_net.state_dict())
        self.target_net.eval()

        # 최적화 함수 정의
        self.optimizer = optim.Adam(self.main_net.parameters(), lr=self.alpha)
        # Experience replay를 위한 buffer 정의
        self.buffer = ReplayBuffer(500)

    # state의 인덱스가 연속적인 의미를 가지고 있지 않으므로 효과적인 학습을
    # 위해 one-hot encoding을 수행
    def one_hot_state(self, state):
        one_hot_encoded = np.zeros((1, self.state_no))
        one_hot_encoded[0, state] = 1

        return one_hot_encoded
    
    # 학습이 끝난 후 Q-Network에서 Q-value 계산하는 함수
    def get_q_values(self):
        q_values = defaultdict(lambda: [0.0] * self.action_no)
        # 각 state 별 Q-value 계산
        for i in range(self.state_no):
            state = torch.tensor(self.one_hot_state(i)).float()
            q_values[i] = self.main_net(state).tolist()
        return q_values

    # 신경망 최적화 모델
    def optimize_model(self):
        # Experience replay를 위한 buffer의 크기가 Experience replay에서의 batch size보다 작으면 return
        if len(self.buffer) < self.batch_size:
            return
        
        # Experience replay : 학습을 위해 transition의 랜덤한 batch를 선택
        transitions = self.buffer.sample(self.batch_size)

        # Transpose the batch 
        # - Transition의 배치배열을 배치배열의 Transition으로 전환 
        # :: Transition - ('state', 'action', 'reward', 'next_state')
        batch = Transition(*zip(*transitions))

        # 최종상태가 아닌 state의 mask를 계산 & 배치 elements 연결 
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), dtype=torch.bool)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None]) 
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)

        # Q(s, a) 계산 - target 계산이 아니므로 main_net을 이용
        # - 모델이 Q(s)를 계산하고, 취해지는 action의 열을 선택
        # (main_Q-network에 따라 각각의 batch state에 대해 취해지는 action)
        state_action_values = self.main_net(state_batch).gather(1, action_batch)

        # reward + gamma*maxa(S', a' : w) 계산(target value)
        # Fixed target Q-value : target value이기 때문에 target net을 이용하여 계산 수행
        # 모든 다음 state에 대해 Q(S') 계산 (next_stat_values : maxa(Q(s')))
        # - 'non_final_values'의 다음 기대 action의 값은 older target net에 의해서 계산됨 (가장 높은 reward를 선택)
        # mask를 기반으로 병합되므로, state가 최종일 경우 예상 state 값 or 0을 가질 수 있다.
        next_state_values = torch.zeros(self.batch_size)
        next_state_values[non_final_mask] = self.target_net(non_final_next_states).max(1)[0].detach()
        # target Q values 계산
        target_state_action_values = (next_state_values * self.gamma) + reward_batch

        ## Compute Huber loss
        # 손실함수 정의
        criterion = nn.SmoothL1Loss()
        # 오류 계산 :: (target value - Q(s,a))
        loss = criterion(state_action_values, target_state_action_values)
        ## Optimize the model
        # 최적화를 위해 gradient 0으로 초기화
        self.optimizer.zero_grad()
        # 오류 역전파
        loss.backward()
        for param in self.main_net.parameters():
            param.grad.data.clamp_(-10, 10)
        # gradient 갱신
        self.optimizer.step()

    # DQN 갱신 부분
    def update(self, state, action, reward, next_state, time_step):
      # memory buffer안에 transition 정보 저장
        self.buffer.push(torch.from_numpy(state).float(),
                         torch.tensor(action).reshape((-1, 1)),
                         torch.tensor(reward).reshape((-1, 1)),
                         torch.from_numpy(next_state).float())

        # training interval 마다 신경망 최적화 수행
        if (time_step + 1) % self.training_interval == 0:
            self.optimize_model()
        
        # target update interval 마다 target Q-network를 main Q-network 값으로 update
        if (time_step + 1) % self.target_update_interval == 0:
            self.target_net.load_state_dict(self.main_net.state_dict())

    # DQN epsilon-greedy 정책 - behavior policy(실제 행동 수행)
    def act(self, state):
      # np.random.rand() : 0과 1사이의 균일분포에서 action_no(4) 크기의 난수 matrix
        # 숫자가 epsilon보다 작을 경우 랜덤하게 action을 선택
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.action_no)
        # 숫자가 epsilon보다 크거나 같을 경우 q-value을 최대화하는 action을 선택
        else:
            with torch.no_grad():
              state  = torch.from_numpy(state).float() # 형식을 맞춰주기 위해 tensor형식으로 바꿈
              q_values = self.main_net(state)
              action = torch.argmax(q_values, 1).item() # 선택할 수 있는 1x48가지 경우에서 1차원으로 action을 빼내기
              # torch.argmax(q_values, 1)의 output은 tensor([~]) 형태이므로 안의 내용을 빼오려면 .item()을 써야함.
        return action


이전 과제와 동일하게 OpenAI Gym에서의 Cliff Walking 환경을 로드하고, 주어진 Q-value에서 greedy policy를 출력하는 QtoPolicy Class를 정의합니다.


In [ ]:
env = CliffWalkingEnv() # Cliff Walking 환경 load
policy = QtoPolicy() # Q-value의 greedy policy를 출력하는 OtoPolicy 클래스 정의

DQN Class를 정의하고 5000 episode 동안 학습을 수행합니다.

DQN에서의 training interval 및 target Q-network update interval을 위해 episode와 관계없는 time-step을 사용하여 DQN class에서 활용할 수 있게 해줍니다.

In [ ]:
agent_DQN = DQN() # DQN 클래스 정의
time_step = 0
# 5000 episode 동안 학습 수행
for ep in tqdm(range(5000)):
    done = False
    # reset() : 상태 초기화
    state = env.reset()

    state = agent_DQN.one_hot_state(state) # state 원 핫 인코드 수행
    action = agent_DQN.act(state) # DQN action 고르기 (epsilon-greedy 정책 - behavior policy)
    # print(action)

    ep_reward = 0 # 에피소드 한 번의 reward
    ep_steps = 0
    while not done:
        # 알고리즘을 통해 얻어낸 Action을 1 step 수행한 후, 현재 state와 reward 등의 정보를 반환
        # Step()함수의 return : tuple 형태의 (next_state, reward, done, info)를 반환
        next_state, reward, done, info = env.step(action)
        reward = reward 
        # state one-hot encoding 수행
        next_state = agent_DQN.one_hot_state(next_state)

        # 앱실론 greedy 정책으로 다음 action choose
        next_action = agent_DQN.act(next_state)

        # 가치함수 update - target net 이용
        agent_DQN.update(state, action, reward, next_state, time_step)
        time_step = time_step + 1

        ep_reward+=reward # episode 별 reward 계산
        state = next_state # 상태 update
        action = next_action # action update
        ep_steps = ep_steps + 1

  0%|          | 0/5000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:912: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
100%|██████████| 5000/5000 [13:33<00:00,  6.14it/s]


학습된 Q-value를 이용하여 학습된 정책을 출력합니다.

In [ ]:
print('Learned policy by DQN')
policy.printPolicy(agent_DQN.get_q_values())

Learned policy by DQN
init policy= [2 2 2 2 2 2 2 1 2 2 2 2 2 1 2 2 2 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 2 2
 1 1 1 1 1 1 1 1 1 1 1]
value = [-89.07472229003906, -89.05058288574219, -89.07815551757812, -88.93586730957031, -89.28718566894531, -89.19509887695312, -89.16865539550781, -88.76815032958984, -88.97096252441406, -89.09000396728516, -89.26597595214844, -89.02621459960938, -88.98213195800781, -89.68174743652344, -88.83232116699219, -89.06253814697266, -88.73747253417969, -89.40100860595703, -88.84500885009766, -89.107421875, -89.46558380126953, -89.17556762695312, -89.125, -89.03252410888672, -89.08116149902344, -88.90898895263672, -89.01132202148438, -89.02355194091797, -89.10859680175781, -89.16790008544922, -89.07036590576172, -89.3904800415039, -89.06970977783203, -89.14940643310547, -89.08528900146484, -89.05450439453125, -89.06951904296875, -74.77819061279297, -71.58032989501953, -69.44672393798828, -67.49089813232422, -71.4187240600586, -74.56720733642578, -73.89748382568

In [ ]:
print('DQN Total reward :')
print(ep_reward)

DQN Total reward :
-398


In [ ]:
# deep sarsa에서 Experience replay를 위한 replay buffer class 생성
# DQNd의 replay buffer에서 next action 정보 추가
Transition = namedtuple('Transition',
                        ('state', 'action', 'reward', 'next_state', 'next_action'))

# replay buffer
# bounded size의 cyclic buffer - 최근에 관찰한 transition을 가지고 있음
class ReplayBuffer(object):

    # 버퍼 초기화
    def __init__(self, capacity):
        self.buffer = deque([],maxlen=capacity)
    
    # 버퍼에 Agent의 경험 정보 추가
    def push(self, *args):
        self.buffer.append(Transition(*args))

    # .sample() 메쏘드 : 학습을 위해 transition의 랜덤한 batch를 선택
    # experience replay에서는 Agent의 경험 정보를 저장하는 replay buffer에서 
    # 경험정보를 sampling하여 신경망 update에 사용
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    # 버퍼 길이 계산
    def __len__(self):
        return len(self.buffer)

Deep Sarsa 알고리즘 class 정의

In [ ]:
# Deep Sarsa 알고리즘 class
class DeepSarsa:
    def __init__(self):
        self.state_no = 48 # 상태 수
        self.action_no = 4 # action 수
        self.alpha = 0.001 # learning rate
        self.gamma = 0.99 # discount vector
        self.epsilon = 0.2 # 앱실론

        self.batch_size = 32  # Experience replay에서의 batch size
        self.training_interval = 10  # Q-Network 학습 interval
        self.target_update_interval = 100  # target Q-Network 학습 interval

         # fixed target Q-value를 위한 main_network, target_network
        self.main_net = DNN(self.state_no, self.action_no)
        self.target_net = DNN(self.state_no, self.action_no)

        # target Q-network를 main Q-network와 동일하게 초기화
        self.target_net.load_state_dict(self.main_net.state_dict())
        self.target_net.eval()

        # 최적화 함수
        self.optimizer = optim.Adam(self.main_net.parameters(), lr=self.alpha)
        # Experience replay를 위한 replay buffer 정의
        self.buffer = ReplayBuffer(500)

    # state의 인덱스가 연속적인 의미를 가지고 있지 않으므로 효과적인 학습을
    # 위해 one-hot encoding을 수행
    def one_hot_state(self, state):
        one_hot_encoded = np.zeros((1, self.state_no))
        one_hot_encoded[0, state] = 1

        return one_hot_encoded
    
    # 학습이 끝난 후 Q-Network에서 Q-value 계산하는 함수
    def get_q_values(self):
        q_values = defaultdict(lambda: [0.0] * self.action_no)
        # 각 state 별 Q-value 계산
        for i in range(self.state_no):
            state = torch.tensor(self.one_hot_state(i)).float()
            q_values[i] = self.main_net(state).tolist()
        return q_values

    # 신경망 최적화 모델
    def optimize_model(self):
        # Experience replay를 위한 buffer의 크기가 Experience replay에서의 batch size보다 작으면 return
        if len(self.buffer) < self.batch_size:
            return
        
        # 학습을 위해 transition의 랜덤한 batch를 선택
        transitions = self.buffer.sample(self.batch_size)

        # Transpose the batch 
        # - Transition의 배치배열을 배치배열의 Transition으로 전환 
        # :: Transition - ('state', 'action', 'reward', 'next_state', 'next_action')
        batch = Transition(*zip(*transitions))

        # 최종상태가 아닌 state의 mask를 계산 & 배치 elements 연결 
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), dtype=torch.bool)
        # 연결
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None]) 
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        next_action_batch = torch.cat(batch.next_action)

        # Q(s, a) 계산 : target value가 아니므로 main net 사용
        # - 모델이 Q(s)를 계산하고, 취해지는 action의 열을 선택
        # (main_Q-network에 따라 각각의 batch state에 대해 취해지는 action)
        state_action_values = self.main_net(state_batch).gather(1, action_batch)  

        # target value 계산 : target net 이용
        # next_state_values : Q(S', A')
        next_state_values = self.target_net(non_final_next_states).gather(1, next_action_batch) ##
        # Compute the expected Q values
        target_state_action_values = (next_state_values * self.gamma) + reward_batch

        ## Compute Huber loss
        # 손실함수 정의
        criterion = nn.SmoothL1Loss()
        # 오류 계산 :: (target value - Q(s,a))
        loss = criterion(state_action_values, target_state_action_values)
        ## Optimize the model
        # 최적화를 위해 gradient 0으로 초기화
        self.optimizer.zero_grad()
        # 오류 역전파
        loss.backward()
        for param in self.main_net.parameters():
            param.grad.data.clamp_(-10, 10)
        # gradient 갱신
        self.optimizer.step()

    # DQN 갱신 부분
    def update(self, state, action, reward, next_state, next_action, time_step):
      # memory buffer안에 transition 저장
        self.buffer.push(torch.from_numpy(state).float(),
                         torch.tensor(action).reshape((-1, 1)),
                         torch.tensor(reward).reshape((-1, 1)),
                         torch.from_numpy(next_state).float(),
                         torch.tensor(next_action).reshape((-1, 1)))
                         

        # training interval 마다 신경망 최적화 수행
        # Deep Sarsa에서는 앱실론 greedy로 예측한 다음 action의 정보가 있어야 갱신 가능
        if (time_step + 1) % self.training_interval == 0:
            self.optimize_model()
        
        # target update interval 마다 target network를 main network의 값으로 update
        if (time_step + 1) % self.target_update_interval == 0:
            self.target_net.load_state_dict(self.main_net.state_dict())

    # DQN epsilon-greedy 정책
    def act(self, state):
      # np.random.rand() : 0과 1사이의 균일분포에서 action_no(4) 크기의 난수 matrix
        # 숫자가 epsilon보다 작을 경우 랜덤하게 action을 선택
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.action_no)
        # 숫자가 epsilon보다 크거나 같을 경우 q-value을 최대화하는 action을 선택
        else:
            with torch.no_grad():
              # 내가 작성
              state  = torch.from_numpy(state).float() # 형식을 맞춰주기 위해 tensor형식으로 바꿈
              q_values = self.main_net(state)
              action = torch.argmax(q_values, 1).item() # 선택할 수 있는 1x48가지 경우에서 1차원으로 action을 빼내기
              # torch.argmax(q_values, 1)의 output은 tensor([~]) 형태이므로 안의 내용을 빼오려면 .item()을 써야함.
        return action


Deep Sarsa Class를 정의하고 5000 episode 동안 학습을 수행합니다.

Deep Sarsa에서의 training interval 및 target Q-network update interval을 위해 episode와 관계없는 time-step을 사용하여 DQN class에서 활용할 수 있게 해줍니다.

In [ ]:
agent_DSa = DeepSarsa() # Deep Sarsa 클래스 정의
time_step = 0
# 5000 episode 동안 학습 수행
for ep in tqdm(range(5000)):
    done = False
    # reset() : 상태 초기화 - Step을 실행하다가 epsiode가 끝나서 이를 초기화해서 재시작해야할 때, 초기 State를 반환
    state = env.reset()

    state = agent_DSa.one_hot_state(state) # state 원핫인코드 수행
    action = agent_DSa.act(state) # epsilon greedy policy로 action 고르기
    # print(action)

    ep_reward = 0
    ep_steps = 0
    while not done:
        # 알고리즘을 통해 얻어낸 Action을 1 step 수행한 후, 현재 state와 reward 등의 정보를 반환
        # Step()함수의 return : tuple 형태의 (next_state, reward, done, info)를 반환
        next_state, reward, done, info = env.step(action)
        reward = reward
        # state one-hot encoding 수행
        next_state = agent_DSa.one_hot_state(next_state)

        # 앱실론 greedy 정책으로 다음 action choose
        next_action = agent_DSa.act(next_state)

        # 가치함수 update - Sarsa에서는 다음 action 정보가 필요
        agent_DSa.update(state, action, reward, next_state, next_action, time_step)
        time_step = time_step + 1

        ep_reward+=reward
        state = next_state # 상태 update
        action = next_action # action update
        ep_steps = ep_steps + 1

100%|██████████| 5000/5000 [03:08<00:00, 26.56it/s]


학습된 Q-value를 이용하여 학습된 정책을 출력합니다.

In [ ]:
print('Learned policy by Deep Sarsa')
policy.printPolicy(agent_DSa.get_q_values())

Learned policy by Deep Sarsa
init policy= [1 2 1 1 1 1 1 1 2 1 1 2 2 1 1 2 1 1 2 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 0
 1 1 1 1 1 1 1 1 1 1 1]
value = [-55.829227447509766, -55.10524368286133, -56.29279708862305, -55.92826461791992, -54.66456604003906, -52.688968658447266, -52.884456634521484, -51.41745376586914, -49.75448226928711, -50.210960388183594, -49.3790397644043, -49.28713607788086, -56.17568588256836, -56.72223663330078, -55.17209243774414, -53.9659538269043, -54.203346252441406, -53.01736068725586, -51.52580642700195, -50.77650451660156, -49.760494232177734, -49.87345504760742, -49.49148178100586, -47.81624221801758, -55.50003433227539, -54.16746139526367, -53.3201789855957, -52.9602165222168, -52.453773498535156, -51.424720764160156, -50.747413635253906, -49.953731536865234, -49.25376510620117, -48.68437194824219, -48.53280258178711, -47.27762985229492, -56.830406188964844, -48.062984466552734, -45.29643249511719, -44.869293212890625, -43.04885482788086, -46.00102615356445, -

In [ ]:
print('Deep Sarsa Total reward :')
print(ep_reward)

Deep Sarsa Total reward :
-36


In [ ]:
env.close()